In [1]:
from bs4 import BeautifulSoup
import requests
import re
from datetime import date
from datetime import datetime, timedelta
import json 
import locale

In [2]:
front_page = 'https://www.estrepublicain.fr/sante/coronavirus'

In [3]:
def fetch_links(source_website):
    """this function fetches links from the front page
    and returns a list"""
    
    list_of_links = []
    
    #pagination on the website follows a simple pattern
    #'https://www.estrepublicain.fr/sante/coronavirus?page=NUMBER_OF_PAGE
    #thus, we run a loop 
    
    for i in range(101): 
        url = source_website + '?page=' + str(i)
        page = requests.get(url)    
        data = page.text
        soup = BeautifulSoup(data)
        
        #catching bugs if a link doesn't contain a substring
        for link in soup.find_all('a'):
            try:
                if link.get('href').startswith('/sante/2'):
                    list_of_links.append('https://www.estrepublicain.fr'+link.get('href'))
            except AttributeError:
                continue
    return list_of_links

In [309]:
big_list = fetch_links('https://www.estrepublicain.fr/sante/coronavirus')

In [328]:
big_list[0]

'https://www.estrepublicain.fr/sante/2021/10/14/covid-19-le-pass-sanitaire-ne-sera-pas-eternel-mais-il-va-etre-prolonge'

In [ ]:
with open('list_of_links.txt', 'w') as f:
    for item in big_list:
        f.write("%s\n" % item)

In [18]:
def grab_info(link):
    """this function scrapes information
    from a single web page, including text, title, and date"""
    
    page_single = requests.get(link).text
    soup_single = BeautifulSoup(page_single)
    
    #fetching the date from the link string
    date_from_link = link.split('/sante/')[1].split('/')
    date_fin = '/'.join(date_from_link[:3])
    
    #putting together all the text from the page
    text = []
    for i in soup_single.find_all('p', class_=None):
        text.append(i.get_text())
    text = text[1:]
    text = ' '.join(text)
    
    #fetching the title
    #two options depending on the page structure
    try:
        title = soup_single.find('title').get_text()
    except:
        title = soup_single.find("span",{"class":"headline"}).next_sibling.lstrip()
    
    dictionary = {'date': date_fin,
               'summary':'',
               'text':text,
               'title':title,
               'topic':'',
               'url': link}
    return dictionary

#json.dumps(dictionary, indent = 6, ensure_ascii=False)

In [19]:
list_of_dic = []
with open("list_of_links.txt", "r") as a_file:
    for line in a_file:
        stripped_line = line.strip()
        list_of_dic.append(grab_info(stripped_line))

In [22]:
#ensure_ascii=False makes sure that french special characters are read correctly

with open('lest_republicain.json', 'w') as f:
    json.dump(list_of_dic, f, ensure_ascii=False)